In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from typing import List, Tuple, Optional, Union
from uncertainty_testbed.uncertainty_models.mc_dropout import MCDropoutLayer, MCDropoutKerasClassification
from uncertainty_testbed.utilities.functions import safe_logodds

2022-05-03 17:22:32.123087: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-03 17:22:32.123119: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train = X_train.reshape(60000, 28*28)
X_train = X_train.astype("float32")/255
X_test = X_test.reshape(10000, 28 * 28)
X_test = X_test.astype("float32") / 255
print(X_train.shape, y_train.shape)
print(type(X_train), type(y_train))
print(y_train.min(), y_train.max(), y_train.mean())
print(type(X_train[0, 0]), type(y_train[0]))

layers = (
    MCDropoutLayer(0.25),
    keras.layers.Dense(300, activation="relu"),
    MCDropoutLayer(0.25),
    keras.layers.Dense(300, activation="relu"),
    MCDropoutLayer(0.25),
    keras.layers.Dense(10, activation="softmax")
)
mcd_classifier = MCDropoutKerasClassification(
    layer_tuple=layers,
    optimizer=keras.optimizers.Nadam(lr=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=("accuracy",),
    name="MCDropout"
)

(60000, 784) (60000,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
0 9 4.4539333333333335
<class 'numpy.float32'> <class 'numpy.uint8'>


/home/nikos/.venv/py3-tf2/lib/python3.8/site-packages/keras/optimizer_v2/nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)
2022-05-03 17:22:34.122216: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-03 17:22:34.122250: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-03 17:22:34.122275: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (xps): /proc/driver/nvidia/version does not exist
2022-05-03 17:22:34.122576: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To e

In [3]:
mcd_classifier.fit(X_train, y_train, epochs=4, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/4
469/469 [==============================] - 4s 8ms/step - loss: 0.3881 - accuracy: 0.8819 - val_loss: 0.2187 - val_accuracy: 0.9332
Epoch 2/4
469/469 [==============================] - 3s 7ms/step - loss: 0.1781 - accuracy: 0.9446 - val_loss: 0.1680 - val_accuracy: 0.9487
Epoch 3/4
469/469 [==============================] - 4s 8ms/step - loss: 0.1391 - accuracy: 0.9558 - val_loss: 0.1372 - val_accuracy: 0.9585
Epoch 4/4
469/469 [==============================] - 4s 8ms/step - loss: 0.1152 - accuracy: 0.9640 - val_loss: 0.1240 - val_accuracy: 0.9631


In [4]:
samples = mcd_classifier.sample_posterior_proba(X_test[1:3, :], n=3)
print("Sampled probabilities: {}".format(' '.join(str(samples))))

Sampled probabilities: [ [ [ 1 . 7 1 4 9 4 8 8 5 e - 0 9   7 . 9 7 2 5 8 1 0 8 e - 0 5   9 . 9 9 9 1 1 4 2 7 e - 0 1   8 . 7 6 3 2 6 4 3 0 e - 0 6 
       1 . 7 5 7 9 9 9 1 6 e - 1 0   7 . 1 5 2 4 2 4 4 3 e - 0 9   3 . 9 7 3 0 7 4 6 7 e - 0 8   2 . 2 2 8 7 2 2 8 1 e - 0 8 
       3 . 9 6 9 3 7 0 9 7 e - 0 8   6 . 3 5 4 2 1 5 7 6 e - 1 1 ] 
     [ 3 . 1 9 5 3 5 6 5 1 e - 0 8   9 . 9 9 5 4 3 8 4 6 e - 0 1   5 . 1 5 3 2 7 8 8 3 e - 0 5   9 . 4 9 6 0 9 8 4 1 e - 0 7 
       1 . 7 3 1 7 3 6 4 8 e - 0 5   1 . 2 0 0 4 5 4 3 4 e - 0 5   4 . 0 3 2 0 1 8 1 1 e - 0 5   2 . 2 0 4 0 6 3 6 4 e - 0 4 
       1 . 1 3 2 8 5 7 2 1 e - 0 4   2 . 2 1 3 1 6 5 6 9 e - 0 7 ] ] 
 
   [ [ 4 . 4 7 9 7 0 4 0 7 e - 0 8   2 . 2 1 3 7 4 4 6 6 e - 0 5   9 . 9 9 9 7 6 8 7 3 e - 0 1   4 . 0 7 3 3 3 4 3 1 e - 0 7 
       2 . 0 6 1 8 1 1 6 6 e - 1 1   2 . 6 8 0 7 4 0 4 3 e - 0 8   4 . 6 0 6 9 1 0 1 2 e - 0 7   5 . 6 9 6 1 7 6 6 9 e - 0 8 
       3 . 7 3 5 4 3 0 4 5 e - 0 8   5 . 4 6 3 2 3 6 2 0 e - 1 2 ] 
     [ 9 . 3 9

In [31]:
for _ in range(10):
    samples = mcd_classifier.sample_posterior_proba(X_test[1:3, :], n=1000)
    ptiles = np.percentile(safe_logodds(samples), (0.05, 0.5, 0.95), axis=0, method='weibull')
    print(ptiles[0,0,2], ptiles[1,0,2], ptiles[2,0,2])

1.5600299835205078 2.812189815044403 3.605694338440895
-0.27596235275268555 3.519556349515915 3.6574083453416826
1.3246748447418213 3.402283947467804 3.8762199726104734
2.07354998588562 3.018944046497345 3.552707969546318
-1.0186008214950562 2.645938789844513 3.1419622914791105
1.214621901512146 2.921909415721893 3.6119294519424434
1.1514525413513184 2.982134782075882 3.6453052971363062
1.9619441032409668 2.767418578863144 3.6957801727056503
2.2287402153015137 2.7808558702468873 3.7384458134174348
2.125295877456665 2.946954675912857 3.403171739697456


In [20]:

post_mean = mcd_classifier.posterior_mean_proba(X_test[1:3, :], n=100)
post_std = mcd_classifier.posterior_stddev_proba(X_test[1:3, :], n=100)
print("Posterior probability mean(sd): {}({})".format(post_mean, post_std))

post_lo_mean = mcd_classifier.posterior_mean_logodds(X_test[1:3, :], n=100)
post_lo_std = mcd_classifier.posterior_stddev_logodds(X_test[1:3, :], n=100)
print("Posterior logodds mean(sd): {}({})".format(post_lo_mean, post_lo_std))


Posterior probability mean(sd): [[7.1735798e-08 1.7713964e-03 9.9787122e-01 3.4183441e-04 1.0854859e-08
  7.2213325e-06 7.6409407e-07 2.4090159e-06 4.9750538e-06 3.2186145e-09]
 [2.0594023e-06 9.9805468e-01 3.3153576e-04 6.8043053e-05 2.7028946e-04
  5.9464477e-05 1.6384850e-04 7.4299827e-04 2.9653570e-04 1.0695091e-05]]([[4.9836439e-07 2.2781279e-03 8.9585679e-03 7.4961209e-03 8.6238934e-08
  2.6178233e-05 7.3982943e-07 4.1200233e-06 5.0262115e-06 4.0180659e-09]
 [5.7345410e-06 4.7011832e-03 8.0828427e-04 3.0706156e-04 1.6635278e-04
  3.0151187e-04 4.6297206e-04 3.5464987e-03 3.0477403e-04 1.3846552e-05]])
Posterior logodds mean(sd): [[-21.18109   -10.38558     9.960042  -11.829664  -24.52874   -18.632458
  -20.422583  -18.250479  -17.77285   -27.337086 ]
 [-15.489634    7.4172645  -9.720194  -12.188994  -10.08276   -11.956151
  -10.824431   -8.5606365  -9.638167  -13.704076 ]]([[3.9980762 2.4667046 2.4900854 3.1074405 4.3469462 3.9101305 4.5167747
  3.327371  3.9357872 4.64983  ]
 [2

In [29]:
# debug
x = X_test[1:3, :]
n = 3
if len(x.shape) == 1:  # if sending in a single row of x
    rows = 1
else:
    rows = x.shape[0]
xx = np.tile(x, (n, 1))
yy = mcd_classifier.model.predict(xx)
yy.shape
if len(yy.shape) == 1:  # if sending in a single row of x and drawing only one sample
    outputs = yy.shape
else:
    outputs = yy.shape[1]
preds = np.reshape(yy, (n, rows, outputs), order='c')
#preds.shape
#preds.mean(axis=0)
#preds[:,1,1]
yy.shape
#preds[:,1,1]
print(yy)
print(preds)
print(preds[:,1,1])

[[1.4778698e-07 1.0569250e-03 9.9889177e-01 4.9480706e-05 2.8968388e-09
  2.8178496e-07 1.0008258e-07 3.3761658e-08 1.2788721e-06 7.5691892e-10]
 [5.4003607e-07 9.9882501e-01 1.3123786e-04 2.0271449e-05 6.2778592e-05
  1.8867104e-05 7.5713842e-06 8.3300844e-04 9.8336022e-05 2.3910959e-06]
 [3.3617372e-08 4.6466477e-05 9.9994707e-01 4.8560314e-06 3.2343248e-09
  2.1421185e-08 4.9854027e-08 2.6634240e-08 1.5430044e-06 1.1077732e-09]
 [7.2723446e-07 9.9921358e-01 4.6354808e-06 5.8113296e-06 7.9623878e-06
  1.9634678e-04 4.3663687e-05 5.0674658e-04 1.9524778e-05 1.1410918e-06]
 [9.8380845e-08 9.6859258e-05 9.9953735e-01 3.4573584e-04 4.7155703e-11
  1.2751868e-05 1.5665944e-07 9.1504253e-09 7.0129622e-06 1.7928635e-10]
 [3.1945319e-06 9.9092042e-01 1.6862231e-04 3.9386796e-04 2.1649522e-03
  7.6904784e-05 9.8084885e-05 5.8066836e-03 2.5601190e-04 1.1120750e-04]]
[[[1.4778698e-07 1.0569250e-03 9.9889177e-01 4.9480706e-05 2.8968388e-09
   2.8178496e-07 1.0008258e-07 3.3761658e-08 1.2788721e-

In [9]:
samples[:,0,1]

array([1.2057129e-05, 9.9714977e-01, 2.4006453e-05], dtype=float32)